In [1]:
# Script for Inference
import torch
from torch import nn
import os
import pickle as pkl
from matplotlib import pyplot as plt
from models.unet import UNet

ckpt_path = 'epoch_990.pth'
data_path = './'
file_list = os.listdir(data_path)
for file_name in file_list:
    if file_name.endswith('.pkl'):
        pkl_path = os.path.join(data_path, file_name)
        
# Load Inference data
with open(pkl_path,'rb') as f:
    data = pkl.load(f)
    

data.keys()



dict_keys(['waypoints', 'frames', 'zforces', 'wrenches'])

In [6]:
import torch.nn.functional as F
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = UNet(2)
state_dict = torch.load(ckpt_path,map_location=device)
model.load_state_dict(state_dict["net"])
model.to(device=device)

print('Use pretrained weights')

for indv_frame in data['frames']:
    indv_frame = torch.tensor(np.expand_dims(np.expand_dims(indv_frame[:,:,0]/256, axis=0), axis=0))
    
    print ("before " , indv_frame.size())
    
    indv_frame = F.interpolate(indv_frame, size=(128, 128), mode='bilinear', align_corners=False).to(torch.float32).to(device)
    
    print ("resize " , indv_frame.size())
    
    indv_frame = torch.tensor(indv_frame[0,:]).to(device)
    
    print (indv_frame.size())
    
    model_input = {"input": indv_frame}
    
    output = model(model_input)
    
    print ("predicted", output.size())
    
    output = output.argmax(1).unsqueeze(1)
    
    plt.imshow(indv_frame[0,:,:].detach().cpu())
    plt.show()
    
    plt.imshow(output[0,0,:,:].detach().cpu())
    plt.show()
    print ('')
    

Use pretrained weights
before  torch.Size([1, 1, 1080, 500])
resize  torch.Size([1, 1, 128, 128])
torch.Size([1, 128, 128])


/home/tianpeng/miniconda3/envs/realsense/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


AssertionError: Torch not compiled with CUDA enabled

In [5]:
torch.cuda.is_available()

False